In [1]:
import requests
import json
import re

In [2]:
def get_lipstick_reviews(link):
    reviews = list()
    pattern = re.compile(r'P\d+')
    product_id = re.search(pattern, string = link).group()
    off_set = 0
    while True:
        api_url = ''.join(["https://api.bazaarvoice.com/data/reviews.json?Filter=ProductId%3A",
                          product_id,
                          "&Sort=Helpfulness%3Adesc&Limit=100&Offset=",
                           str(off_set),
                           "&Include=Products%2CComments&Stats=Reviews&passkey=rwbw526r2e7spptqd2qzbkp7&apiversion=5.4"])
        response = requests.get(api_url)
        results = json.loads(response.content, encoding = 'utf-8')['Results']
        if results:
            for review in results:
                reviews.append(review['ReviewText'])
        else:
            break
        off_set += 100
    return reviews

In [3]:
link = "https://www.sephora.com/product/lip-maestro-P393411?skuId=1441583&icid2=products grid:p393411:product"

In [7]:
# x = get_lipstick_reviews(link)

In [ ]:
# x

In [11]:
def get_text():  
    output = get_lipstick_reviews(link)
    text = ''
    for i in output:
        text += ''.join(i)
    return text

In [12]:
text = get_text()

In [14]:
def get_pos_neg_words():
    def get_words(url):
        import requests
        words = requests.get(url).content.decode('latin-1')
        word_list = words.split('\n')
        index = 0
        while index < len(word_list):
            word = word_list[index]
            if ';' in word or not word:
                word_list.pop(index) #pop删掉的是第几位的数字 pop(index)
            else:
                index+=1
        return word_list

    #Get lists of positive and negative words
    p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
    n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
    positive_words = get_words(p_url)
    negative_words = get_words(n_url)
    return positive_words,negative_words

positive_words,negative_words = get_pos_neg_words()

In [25]:
def do_pos_neg_sentiment_analysis(text,debug=False):
    positive_words,negative_words = get_pos_neg_words()
    from nltk import word_tokenize
    results = list()
    cpos = cneg = lpos = lneg = 0
    for word in word_tokenize(text):
        if word in positive_words:
            if debug:
                print("Positive",word)
            cpos+=1
        if word in negative_words:
            if debug:
                print("Negative",word)
            cneg+=1
    results.append((('Positive Prob: ', cpos/len(word_tokenize(text))),('Negative Prob: ', cneg/len(word_tokenize(text)))))
    return results

In [26]:
do_pos_neg_sentiment_analysis(text,debug=False)

[(('Positive Prob: ', 0.06118371568258744),
  ('Negative Prob: ', 0.01953271906731854))]

In [33]:
# Text- Summerization
def build_summary(text):
    from nltk.tokenize import word_tokenize
    from nltk.tokenize import sent_tokenize
    from nltk.probability import FreqDist
    from nltk.corpus import stopwords
    from collections import OrderedDict
    summary_sentences = []
    candidate_sentences = {}
    candidate_sentence_counts = {}
    
    words = word_tokenize(text)
    
    lowercase_words = [word.lower() for word in words
                      if word not in stopwords.words() and word.isalpha()]
    
    word_frequencies = FreqDist(lowercase_words)
    most_frequent_words = FreqDist(lowercase_words).most_common(20)
    sentences = sent_tokenize(text)
    for sentence in sentences:
        candidate_sentences[sentence] = sentence.lower()
    for long, short in candidate_sentences.items():
        count = 0
        for freq_word, frequency_score in most_frequent_words:
            if freq_word in short:
                count += frequency_score
                candidate_sentence_counts[long] = count   
    sorted_sentences = OrderedDict(sorted(
                        candidate_sentence_counts.items(),
                        key = lambda x: x[1],
                        reverse = True)[:2])
    return sorted_sentences   

In [34]:
summary = '\n'.join(build_summary(text))
print(summary)

Considering the price point it was disappointing.i like the color its so nice but its no super amazing product its just greati got maestro in shade 400 in trial size for free and i was like ok, let me try it and to be honest i really loved it, it is very velvety and matte at the same time, it does not dry out lips and it stays pretty long.
I did receive this product free for review.The pigment was amazing although I do feel like this is something you would have to reapply through out the day but if that doesn’t bother you than go for it because it is worth it the color range is beautiful I received this product for free from influenster to reviewVery creamy and stay on all dayI got this product thanks to @influenster in a campaign and I was really surprised, The formula is super light and soft, it is matte but does not dry out the lips, on the contrary it moisturizes them very well, besides it slides well and its coverage It is complete and like its pigmentation.
